In [1]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForCausalLM
from functions import *
import warnings
warnings.filterwarnings("ignore")

## 1. 读取模型

In [2]:
# 初始化模型
model_name = "chatglm6b"
tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name,trust_remote_code=True).cuda()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

## 2. 定义prompt
这里需要严格定义prompt

In [3]:
prompt1 = """文本总结任务：将一段描述某量化私募的配置逻辑的文本进行总结，找到其使用的因子中，量价类因子和基本面类因子的占比是多少。注意，只说量价类因子占比多少和基本面因子占比多少。

下面是范例:
核心基金经理施恩2020年形成了较为完善的策略框架，近两年主要进行一些因子方面的优化调整。目前仍然是中低频量价因子为核心，占比80%以上，基本面和另类因子大约15%。未来优化方向仍然以各领域的因子挖掘为主，逐步丰富基本面、另类、高频量价因子。子策略层面也主要是核心基金经理施恩过往开发积累的，负责组合优化的研究员主要根据基金经理指示做一些细化的研究工作。 因子层面偏好人工挖掘，对因子的逻辑性有较高要求，机器学习挖掘的因子在5%以内，通常会使用10年以上数据做观测，对近2-3年数据给予更高的权重，每个月都在持续更迭因子，无具体比例统计，入库因子大约1000个，实盘使用500个左右，因子组合以非线性为主，实盘上线了9个子策略模型，暂未使用深度的机器学习。子策略模型信号融合后生产个股截面打分，模型暂未使用半小时以内的预测，日间alpha的贡献较为突出，以3-5天频段为主。2023年新招募了1名研究员负责高频因子挖掘，新的因子和策略大约需要3个月的测试，然后进行小规模实盘，每月进行微幅的因子调整。 选股范围为全市场个股，剔除一些ST股、财务方面有重大风险的、流动性较差的新股等，风险约束方面，由于是空气指增，并未针对特定指数做一些跟踪，主要限制单一个股持仓上限0.5%，平均持仓个股600个左右，单一行业从实盘运行情况来看，不超过20%，通过个股分散的限制，产品不会在行业、概念题材中过度暴露。交易信号在全天分布，年双边换手率70倍左右，并未集中在早盘，策略对交易系统的要求是特别高，交易算法主要使用第三方算法服务商，如卡方。 

根据该段的描述，回答应该为：量价类因子：80%; 基本面因子：15%

接下来，请对下述文本进行总结：
xxxxxx

注意，必须严格按照该格式输出：量价类因子：X%; 基本面因子：X%
"""

## 3. 提取尽调报告中的内容

In [4]:
# 指定文件夹路径
folder_path = '尽调报告'
# 正则表达式，假设“配置逻辑”和“持仓特征”都只出现一次，且顺序固定
regex_pattern = r'配置逻辑(.*?)持仓特征'

开始提取本地的尽调报告中与配置逻辑相关的内容

In [5]:
extracted_contents, name_list = extract_content_from_docx(folder_path, regex_pattern)

使用大语言模型推理，并输出结果，储存在表中

In [6]:
final_df,temp_list = get_answer_from_docx_glm(extracted_contents,prompt1,name_list,model,tokenizer)

In [7]:
final_df # 这个是chatglm的输出结果

,私募名称,因子组成
0,上海泓赫投资管理有限公司,量价类因子：70%; 基本面因子：30%
1,上海象上私募基金管理有限公司,量价类因子：70%; 基本面因子：30%
2,北京涌金春生投资管理有限公司,量价类因子：40%；基本面因子：60%
3,英旷资产管理（上海）有限公司,量价类因子：70%; 基本面因子：30%
